In [1]:
import pandas as pd
from urllib.request import urlopen  
import os.path as osp
import os
import logging
import zipfile
from glob import glob
logging.getLogger().setLevel('INFO')

## Helpers

In [2]:
def download_file(url_str, path):
    try:
        url = urlopen(url_str)
        output = open(path, 'wb')       
        output.write(url.read())
        output.close()
    except ConnectionResetError:
        download_file(url_str, path)
    
def extract_file(archive_path, target_dir):
    zip_file = zipfile.ZipFile(archive_path, 'r')
    zip_file.extractall(target_dir)
    zip_file.close()

## Download the dataset

In [3]:
BASE_URL = 'http://tennis-data.co.uk'
DATA_DIR = "tennis_data"
ATP_DIR = './{}/ATP'.format(DATA_DIR)
WTA_DIR = './{}/WTA'.format(DATA_DIR)

ATP_URLS = [BASE_URL + "/%i/%i.zip" % (i,i) for i in range(2000,2019)]
WTA_URLS = [BASE_URL + "/%iw/%i.zip" % (i,i) for i in range(2007,2019)]
'''
os.makedirs(osp.join(ATP_DIR, 'archives'), exist_ok=True)
os.makedirs(osp.join(WTA_DIR, 'archives'), exist_ok=True)

for files, directory in ((ATP_URLS, ATP_DIR), (WTA_URLS, WTA_DIR)):
    for dl_path in files:
        logging.info("downloading & extracting file %s", dl_path)
        archive_path = osp.join(directory, 'archives', osp.basename(dl_path))
        download_file(dl_path, archive_path)
        extract_file(archive_path, directory)
'''
ATP_FILES = sorted(glob("%s/*.xls*" % ATP_DIR))
WTA_FILES = sorted(glob("%s/*.xls*" % WTA_DIR))

df_atp = pd.concat([pd.read_excel(f) for f in ATP_FILES], ignore_index=True)
df_wta = pd.concat([pd.read_excel(f) for f in WTA_FILES], ignore_index=True)

logging.info("%i matches ATP in df_atp", df_atp.shape[0])
logging.info("%i matches WTA in df_wta", df_wta.shape[0])

/home/lorenzo/miniconda3/envs/datas/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/lorenzo/miniconda3/envs/datas/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/lorenzo/miniconda3/envs/datas/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/lorenzo/miniconda3/envs/datas/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/lorenzo/miniconda3/envs/datas/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/home/lorenzo/miniconda3/envs/datas/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: 

## Your work

### Questions's answers

In [10]:
df_atp.head()

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,UBW,UBL,LBW,LBL,SJW,SJL,MaxW,MaxL,AvgW,AvgL
0,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Enqvist T.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Federer R.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Fromberg R.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


1) Answer

In [8]:
df_atp['Winner'].value_counts()[:3]

Federer R.     1121
Nadal R.        891
Djokovic N.     802
Name: Winner, dtype: int64

2) Answer

In [14]:
df_atp[df_atp.Winner == 'Federer R.']['Winner'].count()

1121

3) Answer

In [23]:
df_atp[(df_atp.Winner == 'Federer R.') & ((df_atp.Date.dt.year == 2016) | (df_atp.Date.dt.year == 2017))]['Winner'].count()

74

4) Answer

In [17]:
def my_func(df):

    s = []

    tmp = {}

    for x in list(set(df.Winner).union(set(df.Loser))):
        tmp[x] = [0, 0]


    for idx, win, los in df[['Winner', 'Loser']].itertuples():    
        try:
            s.append(tmp[win][1] / tmp[win][0])
        except:
            s.append('NotDefined')
        
        tmp[win][0] += 1
        tmp[win][1] += 1
        tmp[los][0] += 1
    
    
    return s

In [18]:
df_atp['win_perc'] = my_func(df_atp[['Winner', 'Loser']])

In [19]:
df_atp['win_perc']

0        NotDefined
1        NotDefined
2        NotDefined
3        NotDefined
4        NotDefined
            ...    
52293      0.664032
52294      0.829876
52295      0.665354
52296      0.830052
52297      0.666667
Name: win_perc, Length: 52298, dtype: object